# 1. Exploring the MlFlow Storage to deploy the models

In [3]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://sunrise-mlflow-tracking.mlflow.svc.cluster.local:5080")
client = MlflowClient()

mv = client.get_model_version("cpu-pct-test-5", "1")   # <- pick your model/version
print(mv.source)  # e.g. s3://mlflow/27/3e0850.../artifacts/model


mlflow-artifacts:/27/1bd475a443f94ccaa2085c1c6279216e/artifacts/2025-09-09-14:56:56-hpo-cpu-node-1-pct-model


### Minio fro Mlflow credential

kubectl -n mlflow get secrets | grep -i minio

# Example for Bitnami chart:
kubectl -n mlflow get secret sunrise-minio -o jsonpath='{.data.root-user}' | base64 -d; echo
kubectl -n mlflow get secret sunrise-minio -o jsonpath='{.data.root-password}' | base64 -d; echo

'''

sunrise-minio                   Opaque               2      123d

admin                            user

ySO5ISk7Eq                       password


'''

# 2. KServe


### K server issue #1

Bingo — the ClusterRoleBindings point to the wrong namespace (kubeflow) for the kserve-controller-manager SA. That’s why every can-i came back no and the controller can’t sync caches.

Let’s fix the bindings so they reference the SA in kserve.

1) Recreate the two bindings with the correct subject

(This is safer than JSON-patching indexes; it idempotently overwrites them.)


**Manager -> needs cluster-wide perms**
kubectl create clusterrolebinding kserve-manager-rolebinding \
  --clusterrole=kserve-manager-role \
  --serviceaccount=kserve:kserve-controller-manager \
  -o yaml --dry-run=client | kubectl apply -f -

**Proxy -> used by kube-rbac-proxy sidecar**
kubectl create clusterrolebinding kserve-proxy-rolebinding \
  --clusterrole=kserve-proxy-role \
  --serviceaccount=kserve:kserve-controller-manager \
  -o yaml --dry-run=client | kubectl apply -f -


kubectl -n kserve rollout restart deploy/kserve-controller-manager
kubectl -n kserve get pods -w

**then:**
kubectl -n kserve logs deploy/kserve-controller-manager -c manager --tail=200


for r in \
  "services" \
  "deployments.apps" \
  "virtualservices.networking.istio.io" \
  "httproutes.gateway.networking.k8s.io" \
  "inferenceservices.serving.kserve.io" \
  "trainedmodels.serving.kserve.io" \
  "inferencegraphs.serving.kserve.io"
do
  printf "%-45s " "$r"; \
  kubectl auth can-i --as=system:serviceaccount:kserve:kserve-controller-manager list "$r" --all-namespaces
done
    
######


### #issue 2

this check the problems if it is there becasue the user "system:serviceaccount:kubeflow:kserve-controller-manager" cannot get resource "serviceaccounts" in the namespace "lstm-iqu"
* kubectl -n kubeflow logs deploy/kserve-controller-manager -c manager | grep -i credential^C
* kubectl -n kubeflow get cm inferenceservice-config -o yaml | grep -A5 credentials

note: Because the KServe controller can’t read your ServiceAccount in lstm-iqu, it fails to inject S3 env vars into the storage-initializer, so you get NoCredentialsError

The kserver cannot inject the secrets credenatial and service account in the pods

In [8]:
import torch
print(torch.__version__)

2.8.0+cpu
